In [1]:
# Imaging library: creating images and editing a pixel color
from PIL import Image

In [2]:
# square size relative size
square_size = 3

# actual size in pixels
p = 8 * square_size

# how wide margins are in pixels
buffer = 10

# common colors
white = (255, 255, 255, 255)
black = (0, 0, 0, 255)
red = (255, 0, 0, 255)
green = (0, 255, 0, 255)
blue = (0, 0, 255, 255)

# define a p x p square of types found on board
white_square = [[white for i in range(p)] for j in range(p)]
black_square = [[black for i in range(p)] for j in range(p)]
x_square = [[white for i in range(p)] for j in range(p)]
question_square = [[white for i in range(p)] for j in range(p)]

for i in range(square_size, p-square_size):
    x_square[i][i] = red
    x_square[p-i][i] = red

# implement question mark
question_square[2*square_size][2*square_size] = black
question_square[3*square_size][1*square_size] = black
question_square[4*square_size][1*square_size] = black
question_square[5*square_size][2*square_size] = black
question_square[5*square_size][3*square_size] = black
question_square[4*square_size][4*square_size] = black
question_square[3*square_size][5*square_size] = black
question_square[3*square_size][7*square_size] = black

In [3]:
# Square represents a single square on the nonogram board
class Square():
    white_mark = 0
    black_mark = 1
    x_mark = 2
    question_mark = 3
    
    def __init__(self, row, col):
        # position of the Square globally
        self.row = row
        self.col = col
        # if all work on Square is finished
        self.finished = False
        # no use right now?
        self.color = 0
        # in case you make a guess, so mark which tentatively came first
        self.tentative_number = 0
        # the current Square status
        self.mark = Square.question_mark
        # if black, what position in row_num black square associated with
        self.assoc_index = -1
        # if black, what length in row_num black square associated with
        self.assoc_len = -1
        # insert time
        self.insert_time = -1
        
    def is_finished(self):
        return self.finished
    
    def get_mark(self):
        return self.mark
    
    def is_white(self):
        return self.mark == Square.white_mark
    
    def is_black(self):
        return self.mark == Square.black_mark
    
    def is_x(self):
        return self.mark == Square.x_mark
    
    def is_question(self):
        return self.mark == Square.question_mark
        
    def set_white(self):
        self.color = white
        self.mark = Square.white_mark
        self.finished = True
        
    def set_black(self):
        self.color = black
        self.mark = Square.black_mark
        self.finished = True
        
    def set_x(self):
        self.mark = Square.x_mark
        self.finished = True
        
    def set_question_mark(self):
        self.mark = Square.question_mark
        
    def set_mark(self, input_mark):
        if input_mark == Square.white_mark:
            self.set_white()
        elif input_mark == Square.black_mark:
            self.set_black()
        elif input_mark == Square.x_mark:
            self.set_x()
        elif input_mark == Square.question_mark:
            self.set_question_mark()


In [4]:
def all_finished(board):
    for square_rows in board:
        for square in square_rows:
            if not square.is_question():
                return False
    return True

In [5]:
# draw a p x p pane on pixels
def draw_pane(board, N, M, pixels, height, width, row, col):
    square = board[row][col]
    
    pane = question_square
    if square.mark == Square.white_mark:
        pane = white_square
    elif square.mark == Square.black_mark:
        pane = black_square
    elif square.mark == Square.x_mark:
        pane = x_square
        
    row2 = buffer + 1 + (p+1) * col
    col2 = buffer + 1 + (p+1) * row
    
    for i in range(p):
        for j in range(p):
            pixels[row2 + i, col2 + j] = pane[i][j]
            
    return

# draw the current state of the board
def draw_board(board):
    N = len(board)
    M = len(board[0])
    height = buffer + (p+1) * N + 1 + buffer
    width = buffer + (p+1) * M + 1 + buffer
    
    img = Image.new("RGBA", (width, height,))
    pixels = img.load()
    
    for i in range(height):
        for j in range(width):
            pixels[j,i] = green
            
    for i in range(N):
        for j in range(M):
            draw_pane(board, N, M, pixels, height, width, i, j)
    
    img.show()
    print("width, height = ", width, height)


In [6]:
# deal with input from "input.txt": read in size and row/col numbers
# Format: first line has 2 numbers: number of rows, then number of columns
# next |R| lines: the numbers on the rows, from left to right
# next |C| lines: the numbers on the columns, from top to bottom
def get_input(file_name="input.txt"):
    with open(file_name, 'r') as f:
        N, M = [int(x) for x in f.readline().split()]
        
        row_nums = []
        for i in range(N):
            temp = [int(x) for x in f.readline().split()]
            row_nums.append(temp)
            
        col_nums = []
        for i in range(M):
            temp = [int(x) for x in f.readline().split()]
            col_nums.append(temp)
            
        return row_nums, col_nums

In [7]:
def print_stats(board):
    N = len(board)
    M = len(board[0])
    num_black = 0
    num_x = 0
    num_question = 0
    for i in range(N):
        for j in range(M):
            s = board[i][j]
            if s.get_mark() == Square.black_mark:
                num_black += 1
            elif s.get_mark() == Square.x_mark:
                num_x += 1
            elif s.get_mark() == Square.question_mark:
                num_question += 1
    
    print("N =", N, "M =", M)
    print("Black:", num_black, ", X:", num_x, ", ?:", num_question)
    print("Unknown:", num_question, "/", N*M)

In [8]:
def solve_trivial_row(N, row, row_num):
    L = len(row_num)
    sum1 = sum(row_num)
    covered = L + sum1 - 1
    if covered == N:
        index_row = 0
        for num in row_num:
            for i in range(num):
                row[index_row].set_black()
                index_row += 1
            
            if index_row < N:
                row[index_row].set_x()
                index_row += 1
    elif sum1 == 0:
        for i in range(M):
            row[i].set_x()

def solve_trivial(board, N, M, transpose, row_nums, col_nums):
    for i in range(len(row_nums)):
        solve_trivial_row(M, board[i], row_nums[i])
    
    for i in range(len(col_nums)):
        solve_trivial_row(N, transpose[i], col_nums[i])

In [9]:
# fill in num_black squares in a row, starting at or later than pos
# return -1, -1 if not possible, otehrwise return pos1, pos2
def fill_in_earliest(N, row, pos, num_black):
    if pos < 0 or N <= pos:
        return (-1, -1)
    pos1 = pos
    
    while True:
        # set left endpoint
        while pos1 < N and row[pos1].get_mark() == Square.x_mark:
            pos1 += 1
    
        # set right end point
        pos2 = pos1
        while pos2 < N and row[pos2].get_mark() != Square.x_mark and pos2 - pos1 + 1 < num_black:
            pos2 += 1
        
        # not long enough
        if pos2 >= N:
            return (-1, -1)
        
        # can't fill in here
        if row[pos2].get_mark() == Square.x_mark: #or row[pos2].get_mark() == Square.black_mark:
            pos1 = pos2 + 1
            continue
        
        # not long enough, reached end of array
        if pos2 - pos1 + 1 != num_black:
            return (-1, -1)
        
        # what if next one was black? not good, have to shift up pos1 and pos2
        while pos2 < N-1 and row[pos2 + 1].get_mark() == Square.black_mark:
            pos1 += 1
            pos2 += 1
            
        # maybe the whole bar is black, and it completely covers [pos1, pos2], which is not good
        # if row[pos1 - 1] is black, not good, which means you actually get a bar of length num_black + 1
        
        if 0 < pos1 and row[pos1 - 1].get_mark() == Square.black_mark:
            # pos2 is where the big black bar ends, but [pos1, pos2] needs to be after it
            pos1 = pos2 + 2
            continue
        
        # long enough
        for i in range(pos1, pos2+1):
            row[i].set_black()
            
        return (pos1, pos2)

# find the earliest set of blocks row_num can occupy, as well as the latest
# whatever intersects must be in the board
def naive_overlap_row(N, row, row_num):
    
    # populate early blocks
    earliest = []
    
    for i in range(N):
        s = Square(0, 0)
        s.set_mark(row[i].get_mark())
        earliest.append(s)
        
    which_num_row = [0 for i in range(N)]
    
    index = 1
    pos2 = -2
    for nums in row_num:
        pos1, pos2 = fill_in_earliest(N, earliest, pos2+2, nums)
        
        if pos1 == -1 and pos2 == -1:
            print("impossible!")
            return
        
        
        for i in range(pos1, pos2+1):
            which_num_row[i] = index
        index += 1
    
    # populate latest, but reversed
    latest_reversed = []
    which_num_col_reversed = [0 for i in range(N)]
    
    for i in range(N):
        s = Square(0, 0)
        s.set_mark(row[N - 1 - i].get_mark())
        latest_reversed.append(s)
    
    pos2 = -2
    index = len(row_num)
    for i in range(len(row_num) - 1, -1, -1):
        num = row_num[i]
        pos1, pos2 = fill_in_earliest(N, latest_reversed, pos2+2, num)
        
        if pos1 == -1 and pos2 == -1:
            print("also impossible!")
            return
        
        for i in range(pos1, pos2+1):
            which_num_col_reversed[i] = index
        index -= 1
    
    # populate latest
    which_num_col = []
    for i in range(N-1, -1, -1):
        which_num_col.append(which_num_col_reversed[i])
    
    latest = [latest_reversed[i] for i in range(N-1, -1, -1)]
    
    # compare earliest and latest and check if intersect:
    
    for i in range(N):
        if which_num_row[i] == which_num_col[i] and which_num_row[i] > 0:
            row[i].set_black()
            row[i].assoc_index = which_num_row[i] - 1
            row[i].assoc_len = row_num[which_num_row[i]-1]
    
    return

# overall wrapper function to find overlaps among all rows, cols
def naive_overlap(board, N, M, transpose, row_nums, col_nums):
    # assume everything not finished?
    # use intersections of intervals?
    
    for i in range(N):
        naive_overlap_row(M, board[i], row_nums[i])
    
    for i in range(M):
        naive_overlap_row(N, transpose[i], col_nums[i])

In [10]:
def try_contradiction_valid_row(N, row, row_num):
    '''
    assoc_list = [-1 for i in range(N)]
    for i in range(N):
        s = row[i]
        if s.is_x():
            assoc_list[i] = -2
        else:
            assoc_list[i] = s.assoc_index
    '''
    
    starting = -3
    
    # if its false in all possible interpretations, must be a contradiction
    while starting < N:
        starting += 1
        # make copy as to modify original
        copy = []
        for i in range(N):
            s = Square(0, 0)
            s.set_mark(row[i].get_mark())
            copy.append(s)
    
        # start pos1 at all possible starting points
        pos1 = starting
        for nums in row_num:
            pos1, pos2 = fill_in_earliest(N, copy, pos1+2, nums)
            if pos1 == -1:
                # could not fit in row_num, so false
                break
                
        # find the corresponding actual_row_num
        actual_row_num = []
        low = 0
        while low < N:
            if not row[low].is_black():
                low += 1
            if low >= N:
                break
            high = low
            while high < N and row[high].is_black():
                high += 1
            
            if low != high:
                actual_row_num.append(high-low)
            low = high
        
        if len(actual_row_num) == 0:
            actual_row_num.append(0)
            
        # if row_num == actual_row_num, then putting x in the spot
        # doesn't necessarily result in contradiction
        print("Actual_row_num = ", actual_row_num, " row_num = ", row_num)
        if row_num == actual_row_num:
            print("trueeeeeeeeeeeee")
            return True
        
    # if false under all possible interpretation, contradiction
    print("falseeee")
    return False
    
    
    

def try_contradiction_valid(board, N, M, transpose, row, col, row_nums, col_nums):
    row_squares = board[row]
    col_squares = transpose[col]
    
    row_num = row_nums[row]
    col_num = col_nums[col]
    
    result = try_contradiction_valid_row(M, row_squares, row_num)
    
    if not result:
        return False
    
    result = try_contradiction_valid_row(N, col_squares, col_num)
    
    if not result:
        return False
    
    return True


def try_x_contradiction_rc(board, N, M, transpose, row, col, row_nums, col_nums):
    if row < 0 or row >= N or col < 0 or col >= M:
        return
    
    if board[row][col].is_finished():
        return
    
    print("2row, col = ", row, col)
    
    # is currently a question mark; make it a x_mark and try to derive a contradiction
    
    board[row][col].set_mark(Square.x_mark)
    
    result = try_contradiction_valid(board, N, M, transpose, row, col, row_nums, col_nums)
    
    # if contradiction reached, then it must be black instead of x
    if not result:
        print("yessssssss")
        board[row][col].set_mark(Square.black_mark)
    else:
        board[row][col].set_mark(Square.question_mark)
    

def try_x_contradiction(board, N, M, transpose, row_nums, col_nums):
    for i in range(N):
        for j in range(M):
            try_x_contradiction_rc(board, N, M, transpose, i, j, row_nums, col_nums)

In [11]:
def check_boundary_helper(board, N, M, row_nums):
    for i in range(N):
        row = board[i]
        row_num = row_nums[i]
        if row_num[0] == 0:
            continue
        
        index = 0
        while index < M and board[i][index].is_x():
            index += 1
        if index == M:
            continue
        
        if board[i][index].is_question():
            continue
        
        # must be black
        first_num = row_num[0]
        for k in range(first_num):
            board[i][index].set_mark(Square.black_mark)
            index += 1
        
        # cuz finished, must be followed by an x
        if index < M:
            board[i][index].set_mark(Square.x_mark)
    
    # back side
    for i in range(N):
        row = board[i]
        row_num = row_nums[i]
        if row_num[0] == 0:
            continue
        
        index = M - 1
        while index >= 0 and board[i][index].is_x():
            index -= 1
        if index < 0:
            continue
        if board[i][index].is_question():
            continue
        
        # must be black
        last_num = row_num[-1]
        for k in range(last_num):
            board[i][index].set_mark(Square.black_mark)
            index -= 1
        
        if index >= 0:
            board[i][index].set_mark(Square.x_mark)

# if black square on boundary, fill it in
def check_boundary(board, N, M, transpose, row_nums, col_nums):
    check_boundary_helper(board, N, M, row_nums)
    check_boundary_helper(transpose, M, N, col_nums)

In [12]:
def validate_working_sets_row(N, row, row_num, ws):
    sum1 = sum(row_num)
    sum2 = 0
    for i in range(N):
        if row[i].is_black():
            sum2 += 1
    
    if sum1 == sum2:
        for i in range(N):
            if not row[i].is_black():
                row[i].set_mark(Square.x_mark)
                
        return []
    
    return ws

def validate_working_sets(board, N, M, transpose, row_nums, col_nums, row_ws, col_ws):
    for row, ws in row_ws.items():
        validate_working_sets_row(M, board[row], row_nums[row], ws)
    for col, ws in col_ws.items():
        validate_working_sets_row(N, transpose[col], col_nums[col], ws)
    

In [88]:
def print_small(N, row):
    print("pos_row = [", end="")
    for s in row:
        if s.is_black():
            print("#", end="")
        elif s.is_x():
            print("X", end="")
        elif s.is_question():
            print("?", end="")
    print("]")
    
def fill_in_all(N, row, starting_pos, num_black):
    if pos < 0 or N <= pos:
        return (-1, -1)
    
    pos2 = -1
    for i in range(starting_pos, N):
        pos1 = i
        
        # set left end point
        if row[pos1].is_x():
            continue
        
        # set right end point
        pos2 = pos1
        while pos2 < N and row[pos2].get_mark() != Square.x_mark and pos2 - pos1 + 1 < num_black:
            pos2 += 1

        # not long enough
        if pos2 >= N:
            continue

        # can't fill in here
        if row[pos2].get_mark() == Square.x_mark:
            pos1 = pos2 + 1
            continue

        # not long enough, reached end of array
        if pos2 - pos1 + 1 != num_black:
            return (-1, -1)

        # what if next one was black? not good, have to shift up pos1 and pos2
        while pos2 < N-1 and row[pos2 + 1].get_mark() == Square.black_mark:
            pos1 += 1
            pos2 += 1

        # maybe the whole bar is black, and it completely covers [pos1, pos2], which is not good
        # if row[pos1 - 1] is black, not good, which means you actually get a bar of length num_black + 1

        if 0 < pos1 and row[pos1 - 1].get_mark() == Square.black_mark:
            # pos2 is where the big black bar ends, but [pos1, pos2] needs to be after it
            pos1 = pos2 + 2
            continue

        # long enough
        for i in range(pos1, pos2+1):
            row[i].set_black()

        return (pos1, pos2)
    
    return (-1, -1)

def get_nums(N, row):
    num_list = []
    pos1 = 0
    
    while pos1 < N:
        while pos1 < N and row[pos1].is_x():
            pos1 += 1
        pos2 = pos1
        while pos2 < N and row[pos2].is_black():
            pos2 += 1
        
        if pos1 != pos2:
            num_list.append(pos2 - pos1)
            
        pos1 = pos2 + 1
        print("pos1 , 2 =  ", pos1, pos2)
        
    print("in get_nums, num_list = ", num_list)
    print_small(N, row)
    
    return num_list

def generate_all_possible_row(N, row, row_num):
    temp = []
    starting_pos = -2
    
    while starting_pos < N:
        pos2 = starting_pos
        for nums in row_num:
            pos1, pos2 = fill_in_earliest(N, row, pos2+2, nums)
            
            if pos1 == -1 and pos2 == -1:
                break
                
        num_list = get_nums(N, row)
        if row_num == num_list:
            #print("??????????")
            temp.append(row)
        else:
            print("nah bruh rownum vs num_list = ", row_num, num_list)
            
        starting_pos += 1
    
    return temp

def brute_force(board, N, M, transpose, row_nums, col_nums):
    possible_rows = []
    for i in range(N):
        copy = []
        for j in range(M):
            s = Square(0, 0)
            s.set_mark(board[i][j].get_mark())
            copy.append(s)
        
        temp = generate_all_possible_row(M, copy, row_nums[i])
        
        #print("for row", i, " : number of possible rows is of size", len(temp))
        
        possible_rows.append(temp)
        
        print("row", i, ":")
        for pos_row in temp:
            print_small(M, pos_row)
        print("\n")
    
    
        


In [89]:
# solve the board
def solve(board, N, M, row_nums, col_nums):
    transpose = [[board[i][j] for i in range(N)] for j in range(M)]
    
    row_ws = {}
    for i in range(N):
        row_ws[i] = [0, M-1]
    col_ws = {}
    for i in range(M):
        col_ws[i] = [0, N-1]
    
    # fill in empty, or those that fill an entire row
    solve_trivial(board, N, M, transpose, row_nums, col_nums)
    
    # find intersection of earliest vs latest, those must be colored 
    naive_overlap(board, N, M, transpose, row_nums, col_nums)
    
    # check boundary
    check_boundary(board, N, M, transpose, row_nums, col_nums)
    
    # get working sets of ranges, x-ing out remainder of ? in finished rows
    
    validate_working_sets(board, N, M, transpose, row_nums, col_nums, row_ws, col_ws)
    
    #print("before contradiction!")
    #print_stats(board)
    # create association between numbers and those on boards, adding ranges of black squares
    #try_x_contradiction(board, N, M, transpose, row_nums, col_nums)
    
    print("calling brute force")
    
    brute_force(board, N, M, transpose, row_nums, col_nums)
    
    # fill in x's with contradiction
    
    # fill in black's with contradiction

In [90]:
# First function to be run
def main():
    # read in input
    row_nums, col_nums = get_input("input.txt")
    
    # get size of board
    N = len(row_nums)
    M = len(col_nums)
    
    # create empty board of squares
    board = [[Square(i,j) for j in range(M)] for i in range(N)]
    
    # solve the board
    solve(board, N, M, row_nums, col_nums)
    
    # convert all x's into white
    if all_finished(board):
        print("Completely solved!")
        for square_row in board:
            for square in square_row:
                if square.get_mark() == Square.x_mark:
                    square.set_white()
    
    # render final board graphically
    draw_board(board)
    print_stats(board)

In [91]:
if __name__ == '__main__':
    main()

calling brute force
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [##########XXX##]
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [10, 2]
pos_row = [#######

pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
pos_row = [X##XX#####X####]
pos1 , 2 =   4 3
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
pos_row = [X##XX#####X####]
pos1 , 2 =   4 3
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
pos_row = [X##XX#####X####]
pos1 , 2 =   4 3
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
pos_row = [X##XX#####X####]
pos1 , 2 =   4 3
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
pos_row = [X##XX#####X####]
pos1 , 2 =   4 3
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
pos_row = [X##XX#####X####]
pos1 , 2 =   4 3
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
pos_row = [X##XX#####X####]
pos1 , 2 =   4 3
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
pos_row = [X##XX#####X####]
pos1 , 2 =   4 3
pos1 , 2 =   11 10
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 5, 4]
po

in get_nums, num_list =  [2, 1, 2, 2, 1, 2]
pos_row = [##X#?##?##?#X##]
nah bruh rownum vs num_list =  [2, 1, 2, 2, 2] [2, 1, 2, 2, 1, 2]
pos1 , 2 =   3 2
pos1 , 2 =   5 4
pos1 , 2 =   8 7
pos1 , 2 =   11 10
pos1 , 2 =   13 12
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 1, 2, 2, 1, 2]
pos_row = [##X#?##?##?#X##]
nah bruh rownum vs num_list =  [2, 1, 2, 2, 2] [2, 1, 2, 2, 1, 2]
pos1 , 2 =   3 2
pos1 , 2 =   5 4
pos1 , 2 =   8 7
pos1 , 2 =   11 10
pos1 , 2 =   13 12
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 1, 2, 2, 1, 2]
pos_row = [##X#?##?##?#X##]
nah bruh rownum vs num_list =  [2, 1, 2, 2, 2] [2, 1, 2, 2, 1, 2]
pos1 , 2 =   3 2
pos1 , 2 =   5 4
pos1 , 2 =   8 7
pos1 , 2 =   11 10
pos1 , 2 =   13 12
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 1, 2, 2, 1, 2]
pos_row = [##X#?##?##?#X##]
nah bruh rownum vs num_list =  [2, 1, 2, 2, 2] [2, 1, 2, 2, 1, 2]
row 7 :
pos_row = [##X#?##?##?#X##]


pos1 , 2 =   4 3
pos1 , 2 =   7 6
pos1 , 2 =   9 8
pos1 , 2 =   12 11
pos1 , 2 =   14 

pos_row = [###X##?###?##X#]
nah bruh rownum vs num_list =  [3, 2, 1, 1, 1] [3, 2, 3, 2, 1]
pos1 , 2 =   4 3
pos1 , 2 =   7 6
pos1 , 2 =   11 10
pos1 , 2 =   14 13
pos1 , 2 =   16 15
in get_nums, num_list =  [3, 2, 3, 2, 1]
pos_row = [###X##?###?##X#]
nah bruh rownum vs num_list =  [3, 2, 1, 1, 1] [3, 2, 3, 2, 1]
pos1 , 2 =   4 3
pos1 , 2 =   7 6
pos1 , 2 =   11 10
pos1 , 2 =   14 13
pos1 , 2 =   16 15
in get_nums, num_list =  [3, 2, 3, 2, 1]
pos_row = [###X##?###?##X#]
nah bruh rownum vs num_list =  [3, 2, 1, 1, 1] [3, 2, 3, 2, 1]
pos1 , 2 =   4 3
pos1 , 2 =   7 6
pos1 , 2 =   11 10
pos1 , 2 =   14 13
pos1 , 2 =   16 15
in get_nums, num_list =  [3, 2, 3, 2, 1]
pos_row = [###X##?###?##X#]
nah bruh rownum vs num_list =  [3, 2, 1, 1, 1] [3, 2, 3, 2, 1]
row 9 :


pos1 , 2 =   4 3
pos1 , 2 =   7 6
pos1 , 2 =   9 8
pos1 , 2 =   11 10
pos1 , 2 =   13 12
pos1 , 2 =   16 15
in get_nums, num_list =  [3, 2, 1, 1, 1, 1]
pos_row = [###X##?#?#?#?X#]
nah bruh rownum vs num_list =  [3, 2, 1, 1, 1] [3,

pos1 , 2 =   4 3
pos1 , 2 =   7 6
pos1 , 2 =   9 8
pos1 , 2 =   11 10
pos1 , 2 =   14 13
pos1 , 2 =   16 15
in get_nums, num_list =  [2, 2, 1, 1, 2, 1]
pos_row = [X##X##?#?#?##X#]
nah bruh rownum vs num_list =  [2, 2, 1, 1, 1] [2, 2, 1, 1, 2, 1]
row 11 :


pos1 , 2 =   2 1
pos1 , 2 =   5 4
pos1 , 2 =   7 6
pos1 , 2 =   8 7
pos1 , 2 =   9 8
pos1 , 2 =   10 9
pos1 , 2 =   11 10
pos1 , 2 =   12 11
pos1 , 2 =   13 12
pos1 , 2 =   16 15
in get_nums, num_list =  [1, 1, 1, 1]
pos_row = [#XX#?#???????X#]
nah bruh rownum vs num_list =  [1, 1, 1] [1, 1, 1, 1]
pos1 , 2 =   2 1
pos1 , 2 =   5 4
pos1 , 2 =   7 6
pos1 , 2 =   9 8
pos1 , 2 =   10 9
pos1 , 2 =   11 10
pos1 , 2 =   12 11
pos1 , 2 =   13 12
pos1 , 2 =   16 15
in get_nums, num_list =  [1, 1, 1, 1, 1]
pos_row = [#XX#?#?#?????X#]
nah bruh rownum vs num_list =  [1, 1, 1] [1, 1, 1, 1, 1]
pos1 , 2 =   2 1
pos1 , 2 =   5 4
pos1 , 2 =   7 6
pos1 , 2 =   9 8
pos1 , 2 =   10 9
pos1 , 2 =   11 10
pos1 , 2 =   12 11
pos1 , 2 =   13 12
pos1 , 2 =   

pos_row = [###############]
pos_row = [###############]
pos_row = [###############]
pos_row = [###############]
pos_row = [###############]
pos_row = [###############]
pos_row = [###############]
pos_row = [###############]
pos_row = [###############]
pos_row = [###############]


width, height =  396 396
N = 15 M = 15
Black: 114 , X: 49 , ?: 62
Unknown: 62 / 225


In [81]:
# Input Test Cases!
'''
Possible input in input.txt:
****************************
5 5
5
1
5
1
5
3 1
1 1 1
1 1 1
1 1 1
1 3

****************************

5 5
3
2 2
1 1
2 2
3
3
2 2
1 1
2 2
3

****************************

5 5
5
1 1 1
5
1
1
5
1 1
3
1 1
3

****************************

5 5
5
1
1
1
5
1 1
1 2
1 1 1
2 1
1 1

****************************
Swan
15 15
10 2
9 1
9 4
9 5
2 5 4
1 1 2 4
2 2 2 3
2 1 2 2 2
3 2 1 2 1
3 2 1 1 1
3 2 1 1 1
2 2 1 1 1
1 1 1
2 2
15
4 6 3
5 6 2
5 4 1
4 1 1 1
4 1 1 1 1
5 1 1 1 1
5 1 1 1 1
5 1 1 1 1
6 1 1 1
1 4 1 1 1
1 3 1
4 1 1
5 4 1
1 6 2
15

****************************
Troll Face
40 50
23
19 6
8 3
3 2
3 7 9 2
3 2 4 2 2
2 2
1 1
1 6 2
2 4 10 2
2 7 2 8 1 3
2 2 10 2 10 1 3
5 4 4 2 6 2
2 4 1 2 4 3 2
2 8 2 6 1 2 2
2 1 4 2 3 2 1
2 1 2 4 7 2 1
2 3 4 1 2 4 6 2
4 3 5 4 1 5 2 1 2
1 4 4 2 8 3 3
2 6 2 5 7 3
2 23 7 3
1 5 12 2 6 2 2
1 4 2 2 1 9 2 2
1 26 4 1
1 24 2 2
1 20 2 3 1
1 17 2 3 2
2 5 2 1 1 4 2
2 5 2 2 1 5 1 2
2 22 2 2 3
2 16 4 3
2 4 4
2 1 5 4
2 2 5 4
2 4 4
2 4
2 6
2 8
16
5
8
3 5 9
2 1 1 18
5 2 1 2
3 1 2 1 2
2 211 1 1
2 17 1 1
2 2 14 2 1
2 2 12 1 1
1 2 2 2 7 1 1
2 3 2 12 1 1
2 3 2 1 7 2 1
2 3 2 2 8 1
2 3 2 2 8 1
2 2 3 2 4 2 1
2 1 3 5 7 2 1
2 2 5 13 1
2 1 2 1 4 4 3 2
2 1 2 1 2 4 2 2
2 1 2 4 2 2
2 1 1 7 2 1
2 1 1 2 8 1 2 
2 2 1 2 1 3 2 1 2
2 1 3 1 1 1 3 2 1 2
2 1 2 1 1 3 3 2 2 2 
2 2 1 2 6 2 2 2 2
2 1 2 2 2 6 2 1 2
2 1 4 9 2 2 2
2 1 4 2 8 2 2
1 1 5 1 3 2 2 2
1 1 6 2 3 2 2 2
1 1 4 2 2 2 2 2 2
1 1 3 1 2 3 2 2 2
2 2 3 1 8 2 2
2 1 2 1 10 1 2
2 2 2 2 2 3
2 2 2 2 5 1 2
2 2 1 5 2
2 2 2 3 1
2 1 7 2
3 1 4 2
3 2 2 2
4 2 1 3
3 6 3
3 4 3
2 4
2 3
3 4
6

'''

'\nPossible input in input.txt:\n****************************\n5 5\n5\n1\n5\n1\n5\n3 1\n1 1 1\n1 1 1\n1 1 1\n1 3\n\n****************************\n\n5 5\n3\n2 2\n1 1\n2 2\n3\n3\n2 2\n1 1\n2 2\n3\n\n****************************\n\n5 5\n5\n1 1 1\n5\n1\n1\n5\n1 1\n3\n1 1\n3\n\n****************************\n\n5 5\n5\n1\n1\n1\n5\n1 1\n1 2\n1 1 1\n2 1\n1 1\n\n****************************\nSwan\n15 15\n10 2\n9 1\n9 4\n9 5\n2 5 4\n1 1 2 4\n2 2 2 3\n2 1 2 2 2\n3 2 1 2 1\n3 2 1 1 1\n3 2 1 1 1\n2 2 1 1 1\n1 1 1\n2 2\n15\n4 6 3\n5 6 2\n5 4 1\n4 1 1 1\n4 1 1 1 1\n5 1 1 1 1\n5 1 1 1 1\n5 1 1 1 1\n6 1 1 1\n1 4 1 1 1\n1 3 1\n4 1 1\n5 4 1\n1 6 2\n15\n\n****************************\nTroll Face\n40 50\n23\n19 6\n8 3\n3 2\n3 7 9 2\n3 2 4 2 2\n2 2\n1 1\n1 6 2\n2 4 10 2\n2 7 2 8 1 3\n2 2 10 2 10 1 3\n5 4 4 2 6 2\n2 4 1 2 4 3 2\n2 8 2 6 1 2 2\n2 1 4 2 3 2 1\n2 1 2 4 7 2 1\n2 3 4 1 2 4 6 2\n4 3 5 4 1 5 2 1 2\n1 4 4 2 8 3 3\n2 6 2 5 7 3\n2 23 7 3\n1 5 12 2 6 2 2\n1 4 2 2 1 9 2 2\n1 26 4 1\n1 24 2 2\n1 20 2 3 1\